In [14]:
# Experimentos Colaborativos Default
# Workflow  Data Drifting repair

#require vm con
#   8 vCPU
#  64 GB  memoria RAM


#limpio la memoria
rm( list= ls(all.names= TRUE) )  #remove all objects
gc( full= TRUE )                 #garbage collection

require("data.table")
require("yaml")


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,645080,34.5,1321371,70.6,1321371,70.6
Vcells,1151325,8.8,8388608,64.0,1704650,13.1


In [15]:


#Parametros del script
PARAM  <- list()
PARAM$experimento  <- "DR6210"

PARAM$exp_input  <- "CA6110"

PARAM$variables_intrames  <- TRUE   # atencion eesto esta en TRUEEEEEE

#valores posibles  "ninguno" "rank_simple" , "rank_cero_fijo" , "deflacion"
PARAM$metodo  <- "rank_cero_fijo"
# FIN Parametros del script



In [22]:

#------------------------------------------------------------------------------
#Esta es la parte que los alumnos deben desplegar todo su ingenio
#Agregar aqui sus PROPIAS VARIABLES manuales

AgregarVariables_IntraMes  <- function( dataset )
{
  gc()
  #INICIO de la seccion donde se deben hacer cambios con variables nuevas

  #creo un ctr_quarter que tenga en cuenta cuando los clientes hace 3 menos meses que estan
  dataset[  , ctrx_quarter_normalizado := ctrx_quarter ]
  dataset[ cliente_antiguedad==1 , ctrx_quarter_normalizado := ctrx_quarter * 5 ]
  dataset[ cliente_antiguedad==2 , ctrx_quarter_normalizado := ctrx_quarter * 2 ]
  dataset[ cliente_antiguedad==3 , ctrx_quarter_normalizado := ctrx_quarter * 1.2 ]

  #variable extraida de una tesis de maestria de Irlanda
  dataset[  , mpayroll_sobre_edad  := mpayroll / cliente_edad ]

  #se crean los nuevos campos para MasterCard  y Visa, teniendo en cuenta los NA's
  #varias formas de combinar Visa_status y Master_status
  dataset[ , vm_status01       := pmax( Master_status,  Visa_status, na.rm = TRUE) ]
  dataset[ , vm_status02       := Master_status +  Visa_status ]
  dataset[ , vm_status03       := pmax( ifelse( is.na(Master_status), 10, Master_status) , ifelse( is.na(Visa_status), 10, Visa_status) ) ]
  dataset[ , vm_status04       := ifelse( is.na(Master_status), 10, Master_status)  +  ifelse( is.na(Visa_status), 10, Visa_status)  ]
  dataset[ , vm_status05       := ifelse( is.na(Master_status), 10, Master_status)  +  100*ifelse( is.na(Visa_status), 10, Visa_status)  ]

  dataset[ , vm_status06       := ifelse( is.na(Visa_status), 
                                          ifelse( is.na(Master_status), 10, Master_status), 
                                          Visa_status)  ]

  dataset[ , mv_status07       := ifelse( is.na(Master_status), 
                                          ifelse( is.na(Visa_status), 10, Visa_status), 
                                          Master_status)  ]


  #combino MasterCard y Visa
  dataset[ , vm_mfinanciacion_limite := rowSums( cbind( Master_mfinanciacion_limite,  Visa_mfinanciacion_limite) , na.rm=TRUE ) ]

  dataset[ , vm_Fvencimiento         := pmin( Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE) ]
  dataset[ , vm_Finiciomora          := pmin( Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE) ]
  dataset[ , vm_msaldototal          := rowSums( cbind( Master_msaldototal,  Visa_msaldototal) , na.rm=TRUE ) ]
  dataset[ , vm_msaldopesos          := rowSums( cbind( Master_msaldopesos,  Visa_msaldopesos) , na.rm=TRUE ) ]
  dataset[ , vm_msaldodolares        := rowSums( cbind( Master_msaldodolares,  Visa_msaldodolares) , na.rm=TRUE ) ]
  dataset[ , vm_mconsumospesos       := rowSums( cbind( Master_mconsumospesos,  Visa_mconsumospesos) , na.rm=TRUE ) ]
  dataset[ , vm_mconsumosdolares     := rowSums( cbind( Master_mconsumosdolares,  Visa_mconsumosdolares) , na.rm=TRUE ) ]
  dataset[ , vm_mlimitecompra        := rowSums( cbind( Master_mlimitecompra,  Visa_mlimitecompra) , na.rm=TRUE ) ]
  dataset[ , vm_madelantopesos       := rowSums( cbind( Master_madelantopesos,  Visa_madelantopesos) , na.rm=TRUE ) ]
  dataset[ , vm_madelantodolares     := rowSums( cbind( Master_madelantodolares,  Visa_madelantodolares) , na.rm=TRUE ) ]
  dataset[ , vm_fultimo_cierre       := pmax( Master_fultimo_cierre, Visa_fultimo_cierre, na.rm = TRUE) ]
  dataset[ , vm_mpagado              := rowSums( cbind( Master_mpagado,  Visa_mpagado) , na.rm=TRUE ) ]
  dataset[ , vm_mpagospesos          := rowSums( cbind( Master_mpagospesos,  Visa_mpagospesos) , na.rm=TRUE ) ]
  dataset[ , vm_mpagosdolares        := rowSums( cbind( Master_mpagosdolares,  Visa_mpagosdolares) , na.rm=TRUE ) ]
  dataset[ , vm_fechaalta            := pmax( Master_fechaalta, Visa_fechaalta, na.rm = TRUE) ]
  dataset[ , vm_mconsumototal        := rowSums( cbind( Master_mconsumototal,  Visa_mconsumototal) , na.rm=TRUE ) ]
  dataset[ , vm_cconsumos            := rowSums( cbind( Master_cconsumos,  Visa_cconsumos) , na.rm=TRUE ) ]
  dataset[ , vm_cadelantosefectivo   := rowSums( cbind( Master_cadelantosefectivo,  Visa_cadelantosefectivo) , na.rm=TRUE ) ]
  dataset[ , vm_mpagominimo          := rowSums( cbind( Master_mpagominimo,  Visa_mpagominimo) , na.rm=TRUE ) ]

  #a partir de aqui juego con la suma de Mastercard y Visa
  dataset[ , vmr_Master_mlimitecompra:= Master_mlimitecompra / vm_mlimitecompra ]
  dataset[ , vmr_Visa_mlimitecompra  := Visa_mlimitecompra / vm_mlimitecompra ]
  dataset[ , vmr_msaldototal         := vm_msaldototal / vm_mlimitecompra ]
  dataset[ , vmr_msaldopesos         := vm_msaldopesos / vm_mlimitecompra ]
  dataset[ , vmr_msaldopesos2        := vm_msaldopesos / vm_msaldototal ]
  dataset[ , vmr_msaldodolares       := vm_msaldodolares / vm_mlimitecompra ]
  dataset[ , vmr_msaldodolares2      := vm_msaldodolares / vm_msaldototal ]
  dataset[ , vmr_mconsumospesos      := vm_mconsumospesos / vm_mlimitecompra ]
  dataset[ , vmr_mconsumosdolares    := vm_mconsumosdolares / vm_mlimitecompra ]
  dataset[ , vmr_madelantopesos      := vm_madelantopesos / vm_mlimitecompra ]
  dataset[ , vmr_madelantodolares    := vm_madelantodolares / vm_mlimitecompra ]
  dataset[ , vmr_mpagado             := vm_mpagado / vm_mlimitecompra ]
  dataset[ , vmr_mpagospesos         := vm_mpagospesos / vm_mlimitecompra ]
  dataset[ , vmr_mpagosdolares       := vm_mpagosdolares / vm_mlimitecompra ]
  dataset[ , vmr_mconsumototal       := vm_mconsumototal  / vm_mlimitecompra ]
  dataset[ , vmr_mpagominimo         := vm_mpagominimo  / vm_mlimitecompra ]
 
   #Aqui debe usted agregar sus propias nuevas variables
  #Variable que refleja la cantidad de transacciones generales vs las realizadas x call center
  # El problema es que ccallcenter_transacciones es 0 en muchos casos y genera nulos
  dataset[ , fm_consultas_vs_callcenter  := ccajas_consultas/ccallcenter_transacciones]

   #proporción de transferencias sobre el capital total
   dataset[ , fm_flujo_vs_capital    := (mtransferencias_recibidas - ctransferencias_emitidas)  / mcuentas_saldo ]
   
   dataset[ , fm_deuda_inv    := (mplazo_fijo_dolares+mplazo_fijo_pesos+minversion1_pesos+minversion1_dolares+minversion2)/(mprestamos_personales+mprestamos_prendarios+mprestamos_hipotecarios)]
   dataset[ , fm_desc_tarj    := (mtarjeta_visa_descuentos+mtarjeta_master_descuentos)/(mtarjeta_master_consumo+mtarjeta_visa_consumo)]
   dataset[ , fm_tarj_payr    := (mtarjeta_master_consumo+mtarjeta_visa_consumo)/mpayroll]
   #dataset[ , fm_trans_ctarj  := (ctarjeta_debito_transacciones+ctarjeta_visa_transacciones+ctarjeta_master_transacciones)/(ctarjeta_debito+ctarjeta_visa+ctarjeta_master)]
   dataset[ , fm_cprestamos   := log(1+cprestamos_hipotecarios+cprestamos_prendarios)]
   dataset[ , fm_cinv    := log(1+cinversion1+cinversion2)]
   dataset[ , fm_cinv2    := log(1+cplazo_fijo+cinversion1+cinversion2)]
   dataset[ , fm_cdel    := log(1+deliquencyvisa+deliquencymaster)]
   dataset[ , fm_cseg_cinv    := (cseguro_vida+cseguro_auto+cseguro_vivienda+cseguro_accidentes_personales+ccaja_seguridad/5)+(cinversion1+cinversion2/2)^2]
   dataset[ , fm_pago_trec    := cpagomiscuentas+(ctransferenciasrecibidas^2)]
   dataset[ , fm_ant__edad    := cliente_antigüedad/cliente_edad]
   dataset[ , fm_prop_pp    := mprestamos_personales/(mprestamos_personales+mprestamos_prendarios+mprestamos_hipotecarios)]
   dataset[ , fm_prop_p    := mprestamos_prendarios/(mprestamos_personales+mprestamos_prendarios+mprestamos_hipotecarios)]
   dataset[ , fm_prop_h   := mprestamos_hipotecarios/(mprestamos_personales+mprestamos_prendarios+mprestamos_hipotecarios)]
   dataset[ , fm_prop_inv1    := minversion1_pesos/(minversion1_pesos+minversion2)]
   dataset[ , fm_prop_inv2    := minversion2_pesos/(minversion1_pesos+minversion2)]
   dataset[ , fm_saldo_sueldo    := (Visa_msaldototal+Master_msaldototal)/mpayroll]
   dataset[ , fm_deb_sueldo    := (mautoservicio)/mpayroll]
   dataset[ , fm_ind1    := edad*(mtarjeta_visa_consumo+mtarjeta_master_consumo)/mpayroll]
   dataset[ , fm_ind2    := edad*(mautoservicio)/mpayroll]
   dataset[ , fm_prop_ad    := (mcuenta_corriente_adicional+mcaja_ahorro_adicional)/(mcuenta_corriente_adicional+mcuenta_corriente+mcaja_ahorro+mcaja_ahorro_adicional)]
   dataset[ , fm_chr_saldo    := (ccheques_emitidos_rechazados/mcuentas_saldo)]
   dataset[ , fm_rat_hb    := (chomebanking_transacciones+cmobile_app_trx)/(chomebanking_transacciones+cmobile_app_trx+ccajas_transacciones+ccallcenter_transacciones)]
   dataset[ , fm_ad_saldo    := (Master_madelantopesos+Master_madelantodolares+Visa_madelantopesos+Visa_madelantodolares)/mcuentas_saldo]
   dataset[ , fm_com_sueldo    := (mcomisiones+mcomisiones_otras)/mpayroll]
   dataset[ , fm_cuenta_sueldo    := (ccuenta_corriente+ccaja_ahorro)/mpayroll]
   dataset[ , fm_rat_deb    := mautoservicio/ctarjeta_debito_transacciones]
   dataset[ , fm_rat_visa    := mtarjeta_visa_consumo/ctarjeta_visa_transacciones]
   dataset[ , fm_rat_master    := mtarjeta_master_consumo/ctarjeta_master_transacciones]
   dataset[ , fm_sueldo_tot    := mpayroll2/(mpayroll+mpayroll2)]
   dataset[ , fm_auto_sueldo    := mcuenta_debitos_automaticos/mpayroll]
   dataset[ , fm_tot_deb    := (mcuenta_debitos_automaticos+mpagodeservicios)/mpayroll]
   dataset[ , fm_rat_pmc   := mpagomiscuentas/mpagodeservicios]
   dataset[ , fm_desc_sueldo    := (mcajeros_propios_descuentos+mtarjeta_visa_descuentos+mtarjeta_master_descuentos)/mpayroll]
   dataset[ , fm_visa_desc    := mtarjeta_visa_descuentos/mtarjeta_visa_consumo]
   dataset[ , fm_master_desc    := mtarjeta_master_descuentos/mtarjeta_master_consumo]
   dataset[ , fm_caj_desc    := mcajeros_propios_descuentos/mautoservicio]
   dataset[ , fm_desc_com    := (mcajeros_propios_descuentos+mtarjeta_visa_descuentos+mtarjeta_master_descuentos)/(mcomisiones_mantenimiento+mcomisiones_otras)]
   dataset[ , fm_ind_ahorro    := mforex_buy/mpayroll]
   dataset[ , fm_inv_tot    := (mplazo_fijo_dolares+mplazo_fijo_pesos+minversion1_pesos+minversion1_dolares+minversion2)/mpayroll]
   dataset[ , fm_pf_sueldo    := (mplazo_fijo_dolares+mplazo_fijo_pesos)/mpayroll]
   dataset[ , fm_inv_sueldo    := (minversion1_pesos+minversion1_dolares+minversion2)/mpayroll]
   dataset[ , fm_pres_sueldo    := (mprestamos_personales+mprestamos_prendarios+mprestamos_hipotecarios)/mpayroll]
   dataset[ , fm_ef_flujo    := mextraccion_autoservicio/(mtransferencias_recibidas+mcheques_depositados)]
   dataset[ , fm_rat_rech    := ccheques_emitidos_rechazados/ccheques_emitidos]
   dataset[ , fm_cons_saldo    := (mtarjeta_master_consumo+mtarjeta_visa_consumo)/(Visa_msaldototal+Master_msaldototal)]
   dataset[ , fm_lim_sueldo    := (Master_mlimitecompra+Visa_mlimitecompra)/mpayroll]
   dataset[ , fm_limfin_sueldo    := (Master_mfinanciacion_limite+Visa_mfinanciacion_limite)/mpayroll]
   dataset[ , fm_pago_saldo    := (Visa_mpagado+Master_mpagado)/(Visa_msaldototal+Master_msaldototal)]
   dataset[ , fm_cconsultas    := log(1+ccajas_otras+ccajas_consultas+ccallcenter_transacciones)]
   dataset[ , fm_ind3    := (ctrx_quarter*cliente_antiguedad)/(mcomisiones_mantenimiento+mcomisiones_otras)^2]
   dataset[ , fm_ind4    := (ctrx_quarter*((mtarjeta_visa_descuentos+mtarjeta_master_descuentos+mcajeros_propios_descuentos))/(mcomisiones_mantenimiento+mcomisiones_otras)^2)]
   dataset[ , fm_eftarj    := (cextraccion_autoservicio+ccajas_extracciones)/(ctarjeta_visa_transacciones+ctarjeta_master_transacciones)]
   dataset[ , fm_sueldo_acr    := mpayroll/cpayroll_trx]
   dataset[ , fm_ctransemp    := log(1+cpayroll_trx+cpayroll2_trx)]
   dataset[ , fm_mcauto    := mcuenta_debitos_automaticos/ccuenta_debitos_automaticos]
   dataset[ , fm_indeu    := ((mprestamos_personales+mprestamos_prendarios+mprestamos_hipotecarios)*(mforex_sell)^2)/mpayroll]
   dataset[ , fm_indeu2    := ((mprestamos_personales+mprestamos_prendarios+mprestamos_hipotecarios)*(mforex_sell)^2)/(mpayroll+mplazo_fijo_dolares+mplazo_fijo_pesos+minversion1_pesos+minversion1_dolares+minversion2)]
 
 
 }

In [ ]:
#------------------------------------------------------------------------------
#deflaciona por IPC
#momento 1.0  31-dic-2020 a las 23:59

drift_deflacion  <- function( campos_monetarios )
{
  vfoto_mes <- c( 201901, 201902, 201903, 201904, 201905, 201906,
                  201907, 201908, 201909, 201910, 201911, 201912,
                  202001, 202002, 202003, 202004, 202005, 202006,
                  202007, 202008, 202009, 202010, 202011, 202012,
                  202101, 202102, 202103, 202104, 202105, 202106,
                  202107  )

  vIPC  <- c( 1.9903030878, 1.9174403544, 1.8296186587,
              1.7728862972, 1.7212488323, 1.6776304408,
              1.6431248196, 1.5814483345, 1.4947526791,
              1.4484037589, 1.3913580777, 1.3404220402,
              1.3154288912, 1.2921698342, 1.2472681797,
              1.2300475145, 1.2118694724, 1.1881073259,
              1.1693969743, 1.1375456949, 1.1065619600,
              1.0681100000, 1.0370000000, 1.0000000000,
              0.9680542110, 0.9344152616, 0.8882274350,
              0.8532444140, 0.8251880213, 0.8003763543,
              0.7763107219  )

  tb_IPC  <- data.table( "foto_mes"= vfoto_mes,
                         "IPC" = vIPC )

  dataset[ tb_IPC,
           on= c("foto_mes"),
           (campos_monetarios) :=  .SD * i.IPC ,
           .SDcols = campos_monetarios ]

}

#------------------------------------------------------------------------------

drift_rank_simple  <- function( campos_drift )
{
  for( campo in campos_drift )
  {
    cat( campo, " " )
    dataset[ , paste0(campo,"_rank") :=  (frank( get(campo), ties.method="random") - 1) / ( .N -1 ), by= foto_mes]
    dataset[ , (campo) := NULL ]
  }
}
#------------------------------------------------------------------------------
#El cero se transforma en cero
#los positivos se rankean por su lado
#los negativos se rankean por su lado

drift_rank_cero_fijo  <- function( campos_drift )
{
  for( campo in campos_drift )
  {
    cat( campo, " " )
    dataset[ get(campo) ==0, paste0(campo,"_rank") := 0 ]
    dataset[ get(campo) > 0, paste0(campo,"_rank") :=   frank(  get(campo), ties.method="random")  / .N, by= foto_mes ]
    dataset[ get(campo) < 0, paste0(campo,"_rank") :=  -frank( -get(campo), ties.method="random")  / .N, by= foto_mes ]
    dataset[ , (campo) := NULL ]
  }
}
#------------------------------------------------------------------------------
#------------------------------------------------------------------------------
#Aqui comienza el programa
PARAM$stat$time_start  <- format(Sys.time(), "%Y%m%d %H%M%S")


setwd("D:\\Dropbox\\Austral\\labo12023")   #Establezco el Working Directory

#cargo el dataset donde voy a entrenar
#esta en la carpeta del exp_input y siempre se llama  dataset.csv.gz
dataset_input  <- paste0( "./exp/", PARAM$exp_input, "/dataset.csv.gz" )
dataset  <- fread( dataset_input )

#creo la carpeta donde va el experimento
dir.create( paste0( "./exp/", PARAM$experimento, "/"), showWarnings = FALSE )
setwd(paste0( "./exp/", PARAM$experimento, "/"))   #Establezco el Working Directory DEL EXPERIMENTO

write_yaml( PARAM, file= "parametros.yml" )   #escribo parametros utilizados

#primero agrego las variables manuales
if( PARAM$variables_intrames )  AgregarVariables_IntraMes( dataset )

#ordeno de esta forma por el ranking
setorder( dataset, foto_mes, numero_de_cliente )

#por como armé los nombres de campos, estos son los campos que expresan variables monetarias
campos_monetarios  <- colnames(dataset)
campos_monetarios  <- campos_monetarios[campos_monetarios %like% "^(m|Visa_m|Master_m|vm_m)"]

#aqui aplico un metodo para atacar el data drifting
#hay que probar experimentalmente cual funciona mejor
switch( 
PARAM$metodo,
  "ninguno"        = cat( "No hay correccion del data drifting" ),
  "rank_simple"    = drift_rank_simple( campos_monetarios ),
  "rank_cero_fijo" = drift_rank_cero_fijo( campos_monetarios ),
  "deflacion"      = drift_deflacion( campos_monetarios ) 
)

print(head(dataset[,flujo_vs_capital]))

fwrite( dataset,
        file="dataset.csv.gz",
        logical01= TRUE,
        sep= "," )

#------------------------------------------------------------------------------
PARAM$stat$time_end  <- format(Sys.time(), "%Y%m%d %H%M%S")
write_yaml( PARAM, file= "parametros.yml" )   #escribo parametros utilizados

#dejo la marca final
cat( format(Sys.time(), "%Y%m%d %H%M%S"),"\n",
     file= "zRend.txt",
     append= TRUE  )
